# Data collection

### Load packages/set globals

In [28]:
import tweepy
consumer_key = "51MI8RrYmzO4btCKG4Qb5uqAa"
consumer_secret = "ajpPv3Ag0NvMEQLBIwiPyDyU78BbLZn8IS1gTba4x9ZOHNPMNM"
access_token = "3004471069-VDbNpT9NO0QOtiqKZXkoH5Flv4MArCflIYImXjn"
access_token_secret = "sP6KMjPZXxYAnaae8bOiauLjCVnx8bzWkBk4KU1iZBxdl"

In [29]:
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
# Creating the API object while passing in auth information
api = tweepy.API(auth) 

n = 3 # Tweet count

### Basic API functionality

In [32]:
# Print tweets from my home timeline
for status in tweepy.Cursor(api.home_timeline).items(n):
    # Process a single status
    print(status.text)
#     print(status._json)

Seattle is the Second Coolest City in America. Wait, Second?! https://t.co/jVxey0MVmm https://t.co/qwWleeQcKP
Opioid addicts turn to Kratom for relief https://t.co/yAXITKLdBB https://t.co/00O5qflBxw
Experience aplenty: Seattle returns 14 players, including eight starters, from last year's MLS Cup-winning team.… https://t.co/9z42XmGkX9


In [33]:
# Getting tweets from the New York Times user
for tweet in api.user_timeline(id="nytimes", count=n):
    # printing the text stored inside the tweet object
    print(tweet.text)

When we discard a person’s accumulated possessions, we are throwing out the record of a life https://t.co/hTEyjTS5lJ
Here's where to stream every major '90s comedy https://t.co/25ASoE5QiI
So this happened in our comments section today https://t.co/oOoSEAf0zM


In [34]:
# get tweets by keyword
for tweet in api.search(q="stock", lang="en")[:n]:
    # printing the text stored inside the tweet object
    print(tweet.user.screen_name,"Tweeted:",tweet.text)

QuinnXCII Tweeted: @KyleDeMarco everything is in stock and shipping! Apologize for the confusion
nguyenh49266651 Tweeted: (Download) STOCK ROM TCL S500 (MT6577): https://t.co/mhZ1ncl4al via @YouTube
jayi_wang Tweeted: RT @jayi_wang: https://t.co/KuoZWdWeUT
Real Time Quote &amp; News


# Tweet processing

## Tokenization

### Default Tokenizer

In [63]:
from nltk.tokenize import word_tokenize

In [64]:
# note a default tokenizer doesn't do a good job
tweets = api.search(q="stock", lang="en")
tweet  = tweets[0].text
print(word_tokenize(tweet))

['@', 'mbobbish', '@', 'chrisco_sr', '@', 'POTUS', '@', 'WhiteHouse', 'Neil', 'Gorsuch', 'getting', 'confirmed', 'is', 'good', '.', 'Economy', 'is', 'getting', 'better', ',', 'as', 'the…', 'https', ':', '//t.co/Rlc0docyIE']


### Tweet specific tokenizer

In [65]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [66]:
print(preprocess(tweet))
# ['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']

['@mbobbish', '@chrisco_sr', '@POTUS', '@WhiteHouse', 'Neil', 'Gorsuch', 'getting', 'confirmed', 'is', 'good', '.', 'Economy', 'is', 'getting', 'better', ',', 'as', 'the', '…', 'https://t.co/Rlc0docyIE']


## Stop terms

In [76]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via']

In [77]:
import operator 
import json
from collections import Counter
 
count_all = Counter()
for tweet in tweets[:3]:
    # Create a list with all the terms
    terms_all = [term for term in preprocess(tweet.text) if term not in stop]
    # Update the counter
    count_all.update(terms_all)
    # Print the first 5 most frequent words
    print(tweet.text)
    print(count_all.most_common(5))

@mbobbish @chrisco_sr @POTUS @WhiteHouse Neil Gorsuch getting confirmed is good.  Economy is getting better, as the… https://t.co/Rlc0docyIE
[('getting', 2), ('Economy', 1), ('Gorsuch', 1), ('confirmed', 1), ('Neil', 1)]
@KevinSparkz Is there a stock for the HSD glitch?
[('getting', 2), ('Economy', 1), ('Is', 1), ('Gorsuch', 1), ('stock', 1)]
Me: I don't put much stock in a man that hasn't coached in 10 years.
My wife: UT hasn't had a good coach in 10 years...
#Truth  #GRUMORS
[("hasn't", 2), ('10', 2), ('getting', 2), ('years', 2), ('good', 2)]


In [ ]:
# Count terms only once, equivalent to Document Frequency
terms_single = set(terms_all)
# Count hashtags only
terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]
# Count terms only (no hashtags, no mentions)
terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
              # mind the ((double brackets))
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs